# CSE 5243 - Introduction to Data Mining
## Homework 5: Association Analysis
- Semester: Autumn 2023
- Instructor: Stanley Vernier
- Section: Mon/Wed, 11:10 am - 12:30 pm  
- Student Name: NYJA GOWDA
- Student Email: gowda.32@osu.edu
- Student ID: 500480083

Template Version V1.
***

# Introduction

### Objectives

In this lab, we will use the Instacart Market Basket Analysis dataset (see: https://www.kaggle.com/datasets/psparks/instacart-market-basket-analysis). However, we will use the data to solve a different (completely nonsense) problem.  The dataset CSV files are provided on Carmen.  They are:
- aisles.csv
- departments.csv
- products.csv
- order_products__train_**teb_subset**.csv  (This is a small'ish subset of the full Instacart transaction dataset. If you are brave, have free time on your hands and have a relatively powerful computer, you can try the full dataset, available at the URL listed above.  This is not necessary, though, and be sure to hand in the answers resulting from the smaller dataset.)

The objectives of this assignment are:
- Practice the Association Analysis content we covered this semester.
- Understand “why” the particular topics, techniques, etc., are important from a practical perspective.
- Understand how to choose and use appropriate tools to solve the provided problems.

### Dataset Notes
- The "order_products_..." data file captures the data in "long format". Specifically, every row corresponds to the transaction id and the item. If the specific transaction id has multiple items, there will be multiple rows in the data.
- You can process the data however you like, but it is recommended you convert into a one-hot-encoded data structure. This will allow you to easily use the mlxtend package.

## The Business Problem
- Assume the provided dataset contains all of the transactions for one month for our store (Trader Buck's).  We wish to find association rules that will improve our revenue as follows:
  - Each month, we will choose a particular **"Aisle of the Month"**. We will discount all products in that aisle by 10% each month, with the hope that this will encourage customers to visit our store to purchase those products 5% more frequently, and also purchase other products in other aisles (that are not discounted) more frequently.
  - For simplicity, assume the normal price of all products is $1.00 (a bargain!)
- Practically speaking, we want to come up with **two-item** rules (one antecedent and one consequent: (A -> B)) and choose the ones that best add to our revenues  (based on the rule support, confidence, etc.).

### Proper Answers
- **IMPORTANT:** **Show your work** and **explain it**.  This will help us give partial credit in some cases.

### Collaboration
For this assignment, you must work as an individual. You may informally discuss ideas with classmates, but your work must be your own.

### What You Need to Turn In
- Submit this Jupyter Notebook in .IPYNB format.  Do not "zip" the file.

### Notes
- Feel free to use the **mlxtend** package throughout this assignment.
- If a question asks you to "calculate" the number of "all possible rules", etc., explain the calculation by showing the "formula" you used. This will act as "showing your work".
***

***
# Section: 1 - Get Ready
1A) Load the data, and get it ready for association analysis. Do this with convenient python helper methods as appropriate. Feel free to use the tools given in the example we covered. 
- Suggest: Make the data one-hot encoded.
***

Using !pip install mlxtend since mlxtend oackage was not previously loaded, but other than that all required packages and functions imported. 

In [2]:
import numpy as np
import pandas as pd
!pip install mlxtend
import mlxtend as mlx
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from mlxtend.frequent_patterns import association_rules
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import fpgrowth

In [3]:
# load the aisles data
aisles_df = pd.read_csv('aisles.csv')
display(aisles_df.info())
display(aisles_df.describe())
aisles_df.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134 entries, 0 to 133
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   aisle_id  134 non-null    int64 
 1   aisle     134 non-null    object
dtypes: int64(1), object(1)
memory usage: 2.2+ KB


None

,aisle_id
count,134.000000
mean,67.500000
std,38.826537
min,1.000000
25%,34.250000
50%,67.500000
75%,100.750000
max,134.000000


,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


In [4]:
# load the aisles data
departments_df = pd.read_csv('departments.csv')
display(departments_df.info())
display(departments_df.describe())
departments_df.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   department_id  21 non-null     int64 
 1   department     21 non-null     object
dtypes: int64(1), object(1)
memory usage: 464.0+ bytes


None

,department_id
count,21.000000
mean,11.000000
std,6.204837
min,1.000000
25%,6.000000
50%,11.000000
75%,16.000000
max,21.000000


,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


In [5]:
# load the products data
products_df = pd.read_csv('products.csv')
display(products_df.info())
display(products_df.describe())
products_df.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49688 entries, 0 to 49687
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   product_id     49688 non-null  int64 
 1   product_name   49688 non-null  object
 2   aisle_id       49688 non-null  int64 
 3   department_id  49688 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 1.5+ MB


None

,product_id,aisle_id,department_id
count,49688.000000,49688.000000,49688.000000
mean,24844.500000,67.769582,11.728687
std,14343.834425,38.316162,5.850410
min,1.000000,1.000000,1.000000
25%,12422.750000,35.000000,7.000000
50%,24844.500000,69.000000,13.000000
75%,37266.250000,100.000000,17.000000
max,49688.000000,134.000000,21.000000


,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [6]:
# load the orders data (some columns)
orders_df = pd.read_csv('order_products__train_teb_subset.csv', usecols=['order_id','product_id'])
display(orders_df.info())
display(orders_df.describe())
orders_df.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 163387 entries, 0 to 163386
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype
---  ------      --------------   -----
 0   order_id    163387 non-null  int64
 1   product_id  163387 non-null  int64
dtypes: int64(2)
memory usage: 2.5 MB


None

,order_id,product_id
count,163387.000000,163387.000000
mean,200333.473287,25545.946207
std,115314.521996,14125.982042
min,1.000000,1.000000
25%,99751.000000,13370.000000
50%,202141.000000,25146.000000
75%,299544.000000,37966.000000
max,399998.000000,49686.000000


,order_id,product_id
0,1,49302
1,1,11109
2,1,10246
3,1,49683
4,1,43633


Using the orders_df data we're now going to create a list of products associated with each transaction (order_id). The code is as follows, and a snippet of what it produces:

In [19]:
order_g = orders_df.groupby('order_id')['product_id'].apply(list)
order_g_df = order_g.reset_index(name='product_list')
print(order_g_df)

       order_id                                       product_list
0             1  [49302, 11109, 10246, 49683, 43633, 13176, 472...
1            36  [39612, 19660, 49235, 43086, 46620, 34497, 486...
2            38  [11913, 18159, 4461, 21616, 23622, 32433, 2884...
3            96  [20574, 30391, 40706, 25610, 27966, 24489, 39275]
4            98  [8859, 19731, 43654, 13176, 4357, 37664, 34065...
...         ...                                                ...
15466    399891                         [1999, 3376, 38293, 13829]
15467    399893  [5876, 12341, 32059, 13517, 44753, 11777, 1106...
15468    399897  [43263, 47766, 13627, 40348, 19019, 45005, 254...
15469    399958  [37766, 45007, 38739, 48104, 32666, 6371, 1979...
15470    399998  [26606, 18564, 13176, 45788, 48628, 11109, 369...

[15471 rows x 2 columns]


We are now going to proceed to performing one hot encoding on the data using the TransactionEncoder since we are using the `read the docs` method as it uses a single function to give us the encoding as opposed to hard coded function to extract 0s and 1s for column values. 

In [232]:
from mlxtend.preprocessing import TransactionEncoder

te = mlx.preprocessing.TransactionEncoder()

a = [[y for y in x if pd.notna(y)] for x in order_g.tolist()]
te_array = te.fit(a).transform(a)
ohe_df= pd.DataFrame(te_array, columns=te.columns_)

In [233]:
ohe_df

,1,3,4,9,10,16,23,25,26,28,...,49667,49668,49670,49675,49676,49678,49680,49681,49683,49686
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15466,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
15467,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
15468,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
15469,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


I chose this method for one hot coding through the `read the docs` method. 

***
# Section: 2 - Explore the Data
***

***
## Section: 2.1 - Get the Order and Product Sizes
- Calculate the **number_of_orders** and **number_of_products**.
***

In [24]:
orders_df.nunique()

order_id      15471
product_id    20605
dtype: int64

The number of unique orders: 15,471

The number of unique products: 20,605

***
## Section: 2.2 - Evaluate the Itemset and Rule Size & Complexity
- Calculate the **maximum number of Itemsets** that could be created from the items (without considering the actual transaction data). Show your work.
- Calculate the **maximum number of Rules** that can be created from the items (without considering the actual transaction data). Show your work.
- What do the calculations suggest as a **potential cause of concern**? Hint: Complexity.
- What might you do to manage these concerns?
***

Now we know that d=20605 and those are the number of columns in our one hot encoding. Hence:

1. Max number of itemsets: $2^d$ or $2^{20605}$

In [32]:
2**20605

5285190629580428360225878907041716556025821329634341896139823436131311651838342307372270595988653214541647095896310378208417909819291837566942251817236519177690509594425067727569855824766632108833297978697926204536618470838187029032322574110658400152200006179702752905720841985055254855712422729304040251127003458177603180394740898306446392431899290696907955623638179849520645306218922670104653932406726466158585971897848025055207695319021433979588496719469597087041852525320498102261678509683653869125403011252758256179581573370624276570822088171019605214540683055089296271758558794875557848573124091820711571171707400036795452867629820817338157881048111845348289616050515194784591845662102034611865070177397571023256423562750225942930707511413712159890110670111017200931445249044379356525745886736171864482154624230168549340394848462799492146963395634217146408195377765202706933303784789032796148924967875267804148996462348026003491278534947563779226966581073384586453341177296177845225689837698666

2. Maximum number of Rules= $3^d + 2^{d+1}+1$ or $3^{20605}+2^{20605+1}+1$

In [33]:
3**(20605)+2**(20605)+1

1211862923631624199715235226233686947067242130294397953837127929416091776266605438056391202703388413558516177137631642038638127524379573998406887557572417651735985894019673659252492330311694147517103020701855110461664227724391999671461536017772228738974043137828983170891993801943522810573941038628133411579066565795642315875887247046952159560600652923432305747908836333054877456326325326122034020808729500743730305896916501741481605197818749720406259723181538685961927923859782823931182543070239654959806765141638743755581724951465447763281710010663694010472715476568317857605551671365521627939672503103299503623625912608399648091361282332180752478450329223591303167306493634807324404657403531251758234810475374974635806465999540759857274935764319793813649776304744280056633593830136032118448726810928006018554235013950941762992673210803071154283909612598388736589759541774965405412278595680920238211800504708262581396098047715357602069248068203715775511634427763324984923862659545442982182148280658

***
# Section: 3 - Itemset Generation
***

***
## Section: 3.1 - Revise the Dataset
- If/as appropriate, trim or revise the dataset to make the runtime reasonable.
- Show the results, briefly.
- Explain what you did and why you did it.
***

In [345]:
rows = ohe_df.sample(frac =.25)
rows

,1,3,4,9,10,16,23,25,26,28,...,49667,49668,49670,49675,49676,49678,49680,49681,49683,49686
11838,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
15332,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2956,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
15039,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
15298,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
432,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
232,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
241,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3347,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


I'm randomly sampling the data and taking 25% of the data at random.. The reason for doing that is that sampling creates an unbiased subset of the data, making sure that there are no inherent preferences or support that the rows can carry. Taking 25% or 3868 rows of the data significantly reduces the runtime. 

However, I will continue to use the inital data set and the one hot encoding it produced since to determine `AOTM` I need the right amount of transactions and for rules generation precision is very important. Due to lack of enough records we cannot make the wrong aisle, the `AOTM` as that may result in revenue loss.  

***
## Section: 3.2 - Create Two-Itemsets
- Create a set of about 50 to 100 two-item sets with highest support. Sort them in decreasing order of support.
- Show the results, briefly.
- Explain what you did and why you did it.
***

We are not setting `min_support` of 0.05 as we do not observe a high number of itemset generation for such a high support. For this dataset itemset geenration is more abundant when the support threshold is reduced. Hence, I chose 0.0065 as the `min_support` to have enough itemsets to observe and analyse 2 item itemsets further.  

In [234]:
# Extracting the most frequest itemsets via Mlxtend.
# The length column has been added to increase ease of filtering.
frequent_itemsets1 = apriori(ohe_df, min_support=0.0065, use_colnames=True)
frequent_itemsets1['length'] = frequent_itemsets1['itemsets'].apply(lambda x: len(x))
# printing the frequent itemset
frequent_itemsets1

,support,itemsets,length
0,0.011764,(196),1
1,0.009631,(260),1
2,0.010148,(432),1
3,0.007110,(1940),1
4,0.007239,(2086),1
...,...,...,...
229,0.007433,"(26209, 31717)",2
230,0.011958,"(26209, 47626)",2
231,0.008661,"(26209, 47766)",2
232,0.007045,"(47209, 27966)",2


Now let's apply a condition for 2 items in the itemsets and look at the itemsets generated. We are able to do that by applying a length equality condition for the `itemset` column in the dataframe that contains all possible itemsets with support higher than the minimum support. 

In [182]:
frequent_itemsets2 = apriori(ohe_df, min_support=0.0065, use_colnames=True)
frequent_itemsets2 = frequent_itemsets2[frequent_itemsets2['itemsets'].apply(lambda x: len(x) == 2)]

The folowing is a set of about 52 two-item itemsets with highest support, that are sorted in decreasing order of support.

In [183]:
filtered_df = pd.DataFrame(frequent_itemsets2)
df = filtered_df.sort_values(by='support', ascending=False)
df

,support,itemsets
188,0.025338,"(13176, 21137)"
200,0.018292,"(13176, 47209)"
227,0.018292,"(24852, 47766)"
189,0.017517,"(13176, 21903)"
205,0.015771,"(21137, 24852)"
213,0.015254,"(24852, 21903)"
194,0.014608,"(13176, 27966)"
226,0.014543,"(47626, 24852)"
203,0.014220,"(24852, 16797)"
230,0.011958,"(26209, 47626)"


1. Itemset with highest support = (13176, 21137)
2. Itemset with lowest support = (45007, 21903)

***
# Section: 4 - Generate Rules
- For the two-itemsets created above, create the related rules.
***

Now, we will be using the first itemset we created that had a mix of both 1 and 2 items in the itemset as the `association_rules` function inherently chooses itemsets that have 2 items in their itemset to avoid any code breaking errors that may be caused by a filtered dataset. 

In [314]:
from mlxtend.frequent_patterns import association_rules
rules_ap = association_rules(frequent_itemsets1, metric="confidence", min_threshold=0.05)
rules_ap

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(24852),(4605),0.140392,0.026566,0.008015,0.057090,2.149010,0.004285,1.032373,0.621992
1,(4605),(24852),0.026566,0.140392,0.008015,0.301703,2.149010,0.004285,1.231007,0.549261
2,(4920),(24852),0.030832,0.140392,0.008791,0.285115,2.030856,0.004462,1.202443,0.523745
3,(24852),(4920),0.140392,0.030832,0.008791,0.062615,2.030856,0.004462,1.033906,0.590498
4,(13176),(5876),0.119320,0.027600,0.008467,0.070964,2.571166,0.005174,1.046677,0.693863
...,...,...,...,...,...,...,...,...,...,...
96,(47766),(26209),0.058690,0.045117,0.008661,0.147577,3.271010,0.006013,1.120199,0.737573
97,(47209),(27966),0.055394,0.040915,0.007045,0.127188,3.108568,0.004779,1.098844,0.718086
98,(27966),(47209),0.040915,0.055394,0.007045,0.172196,3.108568,0.004779,1.141099,0.707246
99,(47626),(47766),0.061664,0.058690,0.010794,0.175052,2.982639,0.007175,1.141054,0.708410


Let's filter these rules for the ones that have highest support and a high level of confidence to observe persistent `consequents` and `antecedents`. 

In [315]:
rules = rules_ap

In [317]:
def frozenset_to_int(fs):
    return sum(map(int, fs))

rules['antecedents'] = rules['antecedents'].apply(frozenset_to_int)

In [318]:
def frozenset_to_int(fs):
    return sum(map(int, fs))

rules['consequents'] = rules['consequents'].apply(frozenset_to_int)

***
# Section: 5 - Rule Evaluation
- For the rules created above, find the **Aisle of the Month** and the **set of Products** (that would be given the discount) that would cause the greatest increase in monthly store revenue.
  - This is based on the Business Problem stated at the top of this notebook.
  - Consider:
    - How much will the store's monthly revenue decrease (or increase) due to the change in price for the chosen Products (and its increased sales)?
    - How much will the store's monthly revenue increase (or decrease) due to the increased sales of the associated Products?
***

In [321]:
# Filter the DataFrame based on the list of values
product = products_df[products_df['product_id'].isin(rules['antecedents'])]
product

,product_id,product_name,aisle_id,department_id
4604,4605,Yellow Onions,83,4
4919,4920,Seedless Red Grapes,123,4
5875,5876,Organic Lemon,24,4
8423,8424,Broccoli Crown,83,4
8517,8518,Organic Red Onion,83,4
13175,13176,Bag of Organic Bananas,24,4
16796,16797,Strawberries,24,4
19056,19057,Organic Large Extra Fancy Fuji Apple,24,4
21136,21137,Organic Strawberries,24,4
21902,21903,Organic Baby Spinach,123,4


From this we can see that the AOTM for this subset would be `24`, `16`, `83` or `123`. 

Our focus though is to see which antecedent does not produce a consequent in the same aisle id, because then we would have to discount both the items. Hence let's see when the rules and products data seta are merged on product id where the aisle id is not the same. 

In [330]:
merge_df = pd.merge(rules, products_df, left_on='antecedents', right_on='product_id')
merge_df

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric,product_id,product_name,aisle_id,department_id
0,24852,4605,0.140392,0.026566,0.008015,0.057090,2.149010,0.004285,1.032373,0.621992,24852,Banana,24,4
1,24852,4920,0.140392,0.030832,0.008791,0.062615,2.030856,0.004462,1.033906,0.590498,24852,Banana,24,4
2,24852,16797,0.140392,0.048349,0.014220,0.101289,2.094979,0.007432,1.058907,0.608031,24852,Banana,24,4
3,24852,21137,0.140392,0.083446,0.015771,0.112339,1.346239,0.004056,1.032549,0.299194,24852,Banana,24,4
4,24852,21903,0.140392,0.074462,0.015254,0.108656,1.459211,0.004801,1.038362,0.366095,24852,Banana,24,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,16797,24852,0.048349,0.140392,0.014220,0.294118,2.094979,0.007432,1.217778,0.549222,16797,Strawberries,24,4
97,28204,24852,0.024110,0.140392,0.008209,0.340483,2.425233,0.004824,1.303390,0.602187,28204,Organic Fuji Apple,24,4
98,45066,24852,0.028699,0.140392,0.010148,0.353604,2.518693,0.006119,1.329847,0.620784,45066,Honeycrisp Apple,24,4
99,49683,24852,0.019133,0.140392,0.006528,0.341216,2.430459,0.003842,1.304841,0.600035,49683,Cucumber Kirby,83,4


In [324]:
merge_df2 = pd.merge(rules, products_df, left_on='consequents', right_on='product_id')
merge_df2

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric,product_id,product_name,aisle_id,department_id
0,24852,4605,0.140392,0.026566,0.008015,0.057090,2.149010,0.004285,1.032373,0.621992,4605,Yellow Onions,83,4
1,4605,24852,0.026566,0.140392,0.008015,0.301703,2.149010,0.004285,1.231007,0.549261,24852,Banana,24,4
2,4920,24852,0.030832,0.140392,0.008791,0.285115,2.030856,0.004462,1.202443,0.523745,24852,Banana,24,4
3,8424,24852,0.023205,0.140392,0.006787,0.292479,2.083308,0.003529,1.214958,0.532347,24852,Banana,24,4
4,16797,24852,0.048349,0.140392,0.014220,0.294118,2.094979,0.007432,1.217778,0.549222,24852,Banana,24,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,47626,47766,0.061664,0.058690,0.010794,0.175052,2.982639,0.007175,1.141054,0.708410,47766,Organic Avocado,24,4
97,24852,16797,0.140392,0.048349,0.014220,0.101289,2.094979,0.007432,1.058907,0.608031,16797,Strawberries,24,4
98,24852,28204,0.140392,0.024110,0.008209,0.058471,2.425233,0.004824,1.036496,0.683647,28204,Organic Fuji Apple,24,4
99,24852,45066,0.140392,0.028699,0.010148,0.072284,2.518693,0.006119,1.046981,0.701446,45066,Honeycrisp Apple,24,4


Now we need to focus on the antecedents that do not produce consequents in the same aisle, for that we will take the first merged dataframe and see the rows that do not have a consequent(merge_df2) in the same aisle id as the antecedent(merge_df). 

In [333]:
mval = merge_df[merge_df['aisle_id'] != merge_df2['aisle_id']]
mval

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric,product_id,product_name,aisle_id,department_id
0,24852,4605,0.140392,0.026566,0.008015,0.057090,2.149010,0.004285,1.032373,0.621992,24852,Banana,24,4
12,4605,24852,0.026566,0.140392,0.008015,0.301703,2.149010,0.004285,1.231007,0.549261,4605,Yellow Onions,83,4
13,4920,24852,0.030832,0.140392,0.008791,0.285115,2.030856,0.004462,1.202443,0.523745,4920,Seedless Red Grapes,123,4
16,13176,21137,0.119320,0.083446,0.025338,0.212351,2.544758,0.015381,1.163657,0.689280,13176,Bag of Organic Bananas,24,4
32,8424,24852,0.023205,0.140392,0.006787,0.292479,2.083308,0.003529,1.214958,0.532347,8424,Broccoli Crown,83,4
35,47626,21137,0.061664,0.083446,0.007369,0.119497,1.432018,0.002223,1.040943,0.321511,47626,Large Lemon,24,4
40,8518,47626,0.031608,0.061664,0.006593,0.208589,3.382683,0.004644,1.185650,0.727367,8518,Organic Red Onion,83,4
52,21903,13176,0.074462,0.119320,0.017517,0.235243,1.971531,0.008632,1.151582,0.532425,21903,Organic Baby Spinach,123,4
53,21903,21137,0.074462,0.083446,0.011635,0.156250,1.872458,0.005421,1.086286,0.503429,21903,Organic Baby Spinach,123,4
61,22935,13176,0.032706,0.119320,0.007563,0.231225,1.937858,0.003660,1.145563,0.500330,22935,Organic Yellow Onion,83,4


From the two tables above we can see that `16` was the only aisle id for which the associated product as an antecedent produces a consequent in the same aisle. So it would not make sense to consider `16` to be the aisle of the month. Let's focus our attention to aisles `24`, `83`, and `123`. 

Let's calculate the revenue that purchases in a particular aisle would bring, for that we will see how many products were sold under each aisle, discount those products by 10% and as a result there would be 5% increases in the total products bought in that aisle. 

Formula used: (1-.01(1))x#of products sold + 0.05(#of products sold)x$0.9 

(10% decrease in price evalutes to 0.9 dollars for each product, and there's a 5% growth in no. of purchases)


For aisle `83` the revenue difference would be:

In [347]:
merged_df = pd.merge(orders_df, products_df, left_on='product_id', right_on='product_id')
#merged_df
v=merged_df[merged_df['aisle_id']==83]
print(v)
revenue83 = 0.9*(18021)+0.05*(18021)*0.9
revenuediff = revenue83-18021
revenuediff

        order_id  product_id                       product_name  aisle_id  \
19             1       10246              Organic Celery Hearts        83   
20          2869       10246              Organic Celery Hearts        83   
21          3378       10246              Organic Celery Hearts        83   
22         14119       10246              Organic Celery Hearts        83   
23         17152       10246              Organic Celery Hearts        83   
...          ...         ...                                ...       ...   
162595    362789       43724                            Lettuce        83   
162724    368295        3699    Chard, Green, New England Grown        83   
162874    374148        8175         Baby Bella Whole Mushrooms        83   
163168    388808       45283                 Jumbo White Onions        83   
163203    390368        5163  Premium Super Sweet Corn Bi-Color        83   

        department_id  
19                  4  
20                  4  
21 

-991.1549999999988

Now, let's calculate the revenue difference for aisle `123`:

In [349]:
v=merged_df[merged_df['aisle_id']==123]
print(v)
revenue123 = 0.9*(9376)+0.05*(9376)*0.9
revenuediff = revenue123-9376
revenuediff

        order_id  product_id                   product_name  aisle_id  \
4032          36       43086             Super Greens Salad       123   
4033        3533       43086             Super Greens Salad       123   
4034        7697       43086             Super Greens Salad       123   
4035       24565       43086             Super Greens Salad       123   
4036       27104       43086             Super Greens Salad       123   
...          ...         ...                            ...       ...   
163178    389017       31855  Peeled and Steamed Fava Beans       123   
163214    391219         740                   Celery Heart       123   
163306    395016        6295                Serrano Peppers       123   
163327    396232       43206          Apple Bites Multipack       123   
163352    398058       13616        Steam Fingerling Potato       123   

        department_id  
4032                4  
4033                4  
4034                4  
4035                4  
403

-515.6800000000003

Let's take a look at it for aisle `24`:

In [350]:
v=merged_df[merged_df['aisle_id']==24]
print(v)
revenue83 = 0.9*(17759)+0.05*(17759)*0.9
revenuediff=revenue83-17759
revenuediff

        order_id  product_id                product_name  aisle_id  \
452            1       13176      Bag of Organic Bananas        24   
453           98       13176      Bag of Organic Bananas        24   
454          170       13176      Bag of Organic Bananas        24   
455         1077       13176      Bag of Organic Bananas        24   
456         1325       13176      Bag of Organic Bananas        24   
...          ...         ...                         ...       ...   
162590    362709       16677   USA Florida Juice Oranges        24   
162672    365869       41584                 Cantaloupes        24   
162713    367487       20601  Organic Young Thai Coconut        24   
163183    389147       10360                Dragon Fruit        24   
163296    394810       42411               Young Coconut        24   

        department_id  
452                 4  
453                 4  
454                 4  
455                 4  
456                 4  
...            

-976.744999999999

From this we can see that the biggest loss is incurred by discounting aisle 24 with a difference of $976.74 after discounting the aisle and observing growth. 

We are also interested in calculating how many purchases an antecedent from a particular aisle produces as one purchase of the antecedent produces one purchase in the consequent according to our rules, **we are already using the dataframe where antecedent aisle id is not equal to the aisle id of the consequent.** 

In [339]:
# Filter the DataFrame based on the list of values
product = merged_df[merged_df['product_id'].isin(mval['antecedents'])]
v=product[product['aisle_id']==83]
value_counts = v['product_id'].value_counts()
value_counts

30391    548
45007    524
22935    506
8518     489
24964    485
4605     411
8424     359
49683    296
Name: product_id, dtype: int64

In [340]:
# Filter the DataFrame based on the list of values
product = merged_df[merged_df['product_id'].isin(mval['antecedents'])]
v=product[product['aisle_id']==123]
value_counts = v['product_id'].value_counts()
value_counts

21903    1152
27966     633
39275     542
4920      477
Name: product_id, dtype: int64

In [341]:
# Filter the DataFrame based on the list of values
product = merged_df[merged_df['product_id'].isin(mval['antecedents'])]
v=product[product['aisle_id']==24]
value_counts = v['product_id'].value_counts()
value_counts

24852    2172
13176    1846
47626     954
47209     857
26209     698
37646     372
Name: product_id, dtype: int64

From our rules we know that if the antecedent (1 unit) is sold the consequent (1 unit) will be purchased. But we will also account for the fact that discounting by 10% increased the products sold in that aisle by 5% hence, there will also be a 5% increase in these consequent products bought. 


For example this means that 548 orders are placed for product id 30391 which is aisle 83 which according to our rules gaurantees a purchase in aisle 24 hence, revenue incurred from purchasing product 30391 is 548 purchases of aisle 24 items = 548x1 = $548

**Total for aisle 83:**


| Id | # of units | Revenue | 
|---|---|---|
| 30391| 548 | 548 |
| 45007 | 524  | 524 |
| 22935 | 506 | 506 |
| 8518 | 489  | 489 |
| 24964| 485 | 485 |
| 4605 | 411  | 411 |
| 8424 | 359 | 359 |
| 49683 | 296  | 296 |


Total incremental revenue for AOTM 83 = (548 +524 + 506 + 489 + 485 + 411 + 359 + 296)(1.05) -991.154 = $2,807.75

**Total for aisle 123:**

| Id | # of units | Revenue | 
|---|---|---|
| 21903 | 1152 | 1152 |
| 27966 | 633  | 633 |
| 39275 | 542 | 542 |
| 4920 | 477  | 477 |

Total for AOTM 123 = (1152+633+542+477)(1.05)-515.68 = $2,428.52

**Total for aisle 24:**

| Id | # of units | Revenue | 
|---|---|---|
| 24852 | 2172 | 2172 |
| 13176 | 1846  | 1846 |
| 47626 | 954 | 954 |
| 47209 | 857  | 857 |
| 26209| 698 | 698 |
| 37646 | 372  | 372 |

Total for AOTM 24 = (2172+1846+954+857+857+698+372)(1.05) -976.74= $7167.06

From the above totals, **the highest revenue bringing aisle with not only high in aisle purchases (16,782.255 dollars) but also resulting in purchasing in other aisles ($8,143.8) is `aisle 24`. The highest incremental revenue across all asiles is alsi in aisle 24. Hence, from these consideration we can decide that the `AOTM` should be `aisle 24`.**

***
# Section: 6 - Conclusions
- Write a paragraph on what you discovered or learned from this homework.
***

In my exploration of association analysis through a homework assignment in Python, I delved into the application of the market basket method, specifically employing the Apriori algorithm. This method enables the identification of frequent itemsets in a dataset, revealing patterns of co-occurring items. Through the implementation of the Apriori algorithm, I learned to extract meaningful associations between items in a transactional dataset, emphasizing the importance of support, confidence, and lift as key metrics. The process involved the generation of itemsets meeting predefined support thresholds, subsequent rule creation, and evaluation of the association rules. 

The homework provided valuable insights into parameter tuning for optimal results and highlighted the significance of adjusting the minimum support and confidence levels based on the specific characteristics of the dataset. Additionally, the iterative nature of the Apriori algorithm became apparent, demonstrating its efficiency in progressively identifying frequent itemsets of varying lengths. Overall, this exercise equipped me with a practical understanding of association analysis techniques, empowering me to uncover meaningful connections within transactional data using Python.

***
### END-OF-SUBMISSION
***